In [2]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.0.0


In [4]:
import os

# 设置可使用的 gpu 序号
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# 用来设置是否在特殊情况下在cpu上进行计算
tf.config.set_soft_device_placement = False
# 
tf.config.experimental.set_memory_growth = True
gpus = tf.config.experimental.list_physical_devices('GPU')

print(gpus)

if gpus:
    tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                           [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), len(logical_gpus), 'Logical gpus')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 1 Logical gpus


In [ ]:
# 获取数据
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train) , (x_test, y_test) = fashion_mnist.load_data()
# 定义预处理函数
def data_scale(x, y):
    x = tf.cast(x, tf.float32)
    x = x / 255
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_scale).cache().batch(64)
# next(iter(train_db))# (64,28,28) (64,)

In [ ]:
num_inputs = 784
num_outputs = 10
W = tf.Variable(tf.random.normal(shape=(num_inputs, num_outputs),mean=0, stddev=0.1, dtype=tf.float32))
b = tf.Variable(tf.zeros(num_outputs, dtype=tf.float32))

In [ ]:
X = tf.constant([[1, 2, 3], [4, 5, 6]])

tf.reduce_sum(X, axis=0, keepdims=True), tf.reduce_sum(X, axis=1, keepdims=True),tf.reduce_sum(X)

In [ ]:
def softmax(logits, axis=-1):
    return tf.exp(logits)/tf.reduce_sum(tf.exp(logits), axis, keepdims=True)


In [ ]:
X = tf.random.normal(shape=(2, 5))
X_prob = softmax(X)
X_prob, tf.reduce_sum(X_prob, axis=1)


In [ ]:
# 定义模型
def softmax_net():
    logits = tf.matmul(tf.reshape(X, shape=(-1, W.shape[0])), W) + b
    return softmax(logits)

In [ ]:
# 定义损失函数
def cross_entropy(y_hat, y):
    y = tf.cast(tf.reshape(y, shape=[-1, 1]),dtype=tf.int32)
    y = tf.one_hot(y, depth=y_hat.shape[-1])
    y = tf.cast(tf.reshape(y, shape=[-1, y_hat.shape[-1]]),dtype=tf.int32)
    return -tf.math.log(tf.boolean_mask(y_hat, y)+1e-8)


In [ ]:
def accuracy(y_hat, y):
    return np.mean((tf.argmax(y_hat, axis=1) == y))
